# **Créditos por porcentaje de UMAS**

In [1]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica

# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()

# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, '../../db/datos_json')

# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

# Cargar los archivos JSON y crear DataFrames
for archivo in archivos_json:
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())

# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    # Caso de cuando no son las alfa q
    if nombre.startswith("df_") and "financiamientos" in nombre and "vallarta" in nombre
]

# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

Lista de DataFrames filtrados:


['df_financiamientos_2019_vallarta',
 'df_financiamientos_2020_vallarta',
 'df_financiamientos_2021_vallarta',
 'df_financiamientos_2022_vallarta',
 'df_financiamientos_2023_vallarta',
 'df_financiamientos_2024_vallarta']

## 2022

In [2]:
finance = df_financiamientos_2022_vallarta[['id','ingresos_rango']]
registros_0 = finance[finance['ingresos_rango'] == 0].shape[0]
print(registros_0)
df_2022 = finance.shape
print(df_2022)
#finance_2022_pachuca = finance_2022_pachuca[finance_2022_pachuca['ingresos_rango'] != 0]

5
(1274, 2)


In [3]:
# Convertir la columna 'ingresos_rango' a cadena de texto (str) 
#finance_2022['ingresos_rango'] = finance_2022['ingresos_rango'].astype(str)
finance.loc[:, 'ingresos_rango'] = finance['ingresos_rango'].astype(str)
# Reemplazar los valores de rango con el texto correspondiente utilizando replace
#finance_2022['ingresos_rango'] = finance_2022['ingresos_rango'].replace({
finance.loc[:, 'ingresos_rango'] = finance['ingresos_rango'].replace({
    
    '0': '0',
    '1': '≤ 2.60',
    '2': '2.61-4.00',
    '3': '4.01-6.00',
    '4': '6.01-9.00',
    '5': '9.01-12.00',
    '6': '> 12.00',
})
finance

C:\Users\yoe11\AppData\Local\Temp\ipykernel_18708\2444930754.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['1' '6' '4' ... '3' '4' '5']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  finance.loc[:, 'ingresos_rango'] = finance['ingresos_rango'].astype(str)


,id,ingresos_rango
0,1,≤ 2.60
1,2,> 12.00
2,3,6.01-9.00
3,4,6.01-9.00
4,5,9.01-12.00
...,...,...
1269,1270,2.61-4.00
1270,1271,4.01-6.00
1271,1272,4.01-6.00
1272,1273,6.01-9.00


In [9]:
# Eliminar la fila donde rango_ingresos es '0'
finance = finance[finance['ingresos_rango'] != '0']

# Calcular el conteo de cada rango de ingresos
conteo_ingresos = finance['ingresos_rango'].value_counts(normalize=True).reset_index()
conteo_ingresos.columns = ['rango_ingresos', 'conteo']

# Calcular los porcentajes
conteo_ingresos['porcentajes'] = round(conteo_ingresos['conteo'] * 100, 2)
conteo_ingresos

,rango_ingresos,conteo,porcentajes
0,> 12.00,0.258471,25.85
1,≤ 2.60,0.167061,16.71
2,6.01-9.00,0.165485,16.55
3,2.61-4.00,0.155240,15.52
4,4.01-6.00,0.152088,15.21
5,9.01-12.00,0.101655,10.17


In [10]:
# Definir el diccionario de orden de rangos de ingresos
orden_rangos = {
    '≤ 2.60': 1,
    '2.61-4.00': 2,
    '4.01-6.00': 3,
    '6.01-9.00': 4,
    '9.01-12.00': 5,
    '> 12.00': 6
}

# Añadir una columna para el orden de los rangos
conteo_ingresos['orden'] = conteo_ingresos['rango_ingresos'].map(orden_rangos)
# Ordenar el DataFrame por el orden de los rangos
conteo_ingresos = conteo_ingresos.sort_values(by='orden')
# Eliminar la columna de orden
conteo_ingresos = conteo_ingresos.drop(columns=['orden'])
conteo_ingresos

,rango_ingresos,conteo,porcentajes
1,≤ 2.60,0.167061,16.71
3,2.61-4.00,0.155240,15.52
4,4.01-6.00,0.152088,15.21
2,6.01-9.00,0.165485,16.55
5,9.01-12.00,0.101655,10.17
0,> 12.00,0.258471,25.85


In [11]:
labels = conteo_ingresos['rango_ingresos']
values = conteo_ingresos['porcentajes']
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6']

fig = go.Figure()
fig.add_trace(go.Bar(
        x=labels, 
        y=values, 
        marker_color=colores,
        text=values,  # Texto que se mostrará en las barras (porcentaje)
        textposition='inside',  # Posición del texto (puede ser 'inside' o 'outside')
        #texttemplate='%{text:.1f}%', 
    )
)    
fig.update_layout(
    #barmode='stack', 
    #title='Porcentaje de créditos en UMAS (2022)',
    #title_font_size = 22,
    yaxis=dict(
        #title='M', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y         
    ),  margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    legend=dict(
        yanchor="top",
        y=0.99,
    )
)

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets\graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    # Configuración para desactivar herramientas no deseadas
    config = {
        'displaylogo': False,  # Ocultar el logo de Plotly
        'modeBarButtonsToRemove': [
            'toImage',       # Botón para guardar como imagen
            'select2d',      # Box select
            'lasso2d',       # Lasso select
            'resetScale2d',  # Reset Axes
        ]
    }
    # Exportar el gráfico con configuración
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html', config=config)

guardar_grafico_como_html(fig, 'g_bar_creditos_umas_2022', carpeta='assets\graficas')
fig.show()

<>:31: SyntaxWarning:

invalid escape sequence '\g'

<>:48: SyntaxWarning:

invalid escape sequence '\g'

<>:31: SyntaxWarning:

invalid escape sequence '\g'

<>:48: SyntaxWarning:

invalid escape sequence '\g'

C:\Users\yoe11\AppData\Local\Temp\ipykernel_18708\730601600.py:31: SyntaxWarning:

invalid escape sequence '\g'

C:\Users\yoe11\AppData\Local\Temp\ipykernel_18708\730601600.py:48: SyntaxWarning:

invalid escape sequence '\g'



In [12]:
16.71+15.52+15.21+16.55+10.17+25.85

100.01000000000002

## 2023

In [13]:
finance = df_financiamientos_2023_vallarta[['id','ingresos_rango']]
registros_0 = finance[finance['ingresos_rango'] == 0].shape[0]
print(registros_0)
df_2022 = finance.shape
print(df_2022)
#finance_2022_pachuca = finance_2022_pachuca[finance_2022_pachuca['ingresos_rango'] != 0]

28
(1384, 2)


In [14]:
# Convertir la columna 'ingresos_rango' a cadena de texto (str) 
#finance_2022['ingresos_rango'] = finance_2022['ingresos_rango'].astype(str)
finance.loc[:, 'ingresos_rango'] = finance['ingresos_rango'].astype(str)
# Reemplazar los valores de rango con el texto correspondiente utilizando replace
#finance_2022['ingresos_rango'] = finance_2022['ingresos_rango'].replace({
finance.loc[:, 'ingresos_rango'] = finance['ingresos_rango'].replace({
    
    '0': '0',
    '1': '≤ 2.60',
    '2': '2.61-4.00',
    '3': '4.01-6.00',
    '4': '6.01-9.00',
    '5': '9.01-12.00',
    '6': '> 12.00',
})
finance

C:\Users\yoe11\AppData\Local\Temp\ipykernel_18708\2444930754.py:3: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['6' '5' '6' ... '4' '0' '0']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.



,id,ingresos_rango
0,1,> 12.00
1,2,9.01-12.00
2,3,> 12.00
3,4,> 12.00
4,5,≤ 2.60
...,...,...
1379,1380,≤ 2.60
1380,1381,9.01-12.00
1381,1382,6.01-9.00
1382,1383,0


In [15]:
# Eliminar la fila donde rango_ingresos es '0'
finance = finance[finance['ingresos_rango'] != '0']
# Calcular el conteo de cada rango de ingresos
conteo_ingresos = finance['ingresos_rango'].value_counts().reset_index()
conteo_ingresos.columns = ['rango_ingresos', 'conteo']
# Calcular los porcentajes
total_registros = len(finance)
conteo_ingresos['porcentajes'] = round((conteo_ingresos['conteo'] / total_registros) * 100, 2)
conteo_ingresos

,rango_ingresos,conteo,porcentajes
0,> 12.00,302,22.27
1,6.01-9.00,240,17.70
2,4.01-6.00,235,17.33
3,2.61-4.00,200,14.75
4,≤ 2.60,199,14.68
5,9.01-12.00,180,13.27


In [16]:
# Definir el diccionario de orden de rangos de ingresos
orden_rangos = {
    '≤ 2.60': 1,
    '2.61-4.00': 2,
    '4.01-6.00': 3,
    '6.01-9.00': 4,
    '9.01-12.00': 5,
    '> 12.00': 6
}

# Añadir una columna para el orden de los rangos
conteo_ingresos['orden'] = conteo_ingresos['rango_ingresos'].map(orden_rangos)
# Ordenar el DataFrame por el orden de los rangos
conteo_ingresos = conteo_ingresos.sort_values(by='orden')
# Eliminar la columna de orden
conteo_ingresos = conteo_ingresos.drop(columns=['orden'])
conteo_ingresos

,rango_ingresos,conteo,porcentajes
4,≤ 2.60,199,14.68
3,2.61-4.00,200,14.75
2,4.01-6.00,235,17.33
1,6.01-9.00,240,17.70
5,9.01-12.00,180,13.27
0,> 12.00,302,22.27


In [18]:
labels = conteo_ingresos['rango_ingresos']
values = conteo_ingresos['porcentajes']
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6']

fig = go.Figure()
fig.add_trace(go.Bar(
        x=labels, 
        y=values, 
        marker_color=colores,
        text=values,  # Texto que se mostrará en las barras (porcentaje)
        textposition='inside',  # Posición del texto (puede ser 'inside' o 'outside')
        texttemplate='%{text:.1f}%', 
    )
)    
fig.update_layout(
    #barmode='stack', 
    #title='Porcentaje de créditos en UMAS (2022)',
    #title_font_size = 22,
    yaxis=dict(
        #title='M', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y         
    ),  margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    legend=dict(
        yanchor="top",
        y=0.99,
    )
)

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets\graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    # Configuración para desactivar herramientas no deseadas
    config = {
        'displaylogo': False,  # Ocultar el logo de Plotly
        'modeBarButtonsToRemove': [
            'toImage',       # Botón para guardar como imagen
            'select2d',      # Box select
            'lasso2d',       # Lasso select
            'resetScale2d',  # Reset Axes
        ]
    }
    # Exportar el gráfico con configuración
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html', config=config)

guardar_grafico_como_html(fig, 'g_bar_creditos_umas_2023', carpeta='assets\graficas')
fig.show()

<>:31: SyntaxWarning:

invalid escape sequence '\g'

<>:48: SyntaxWarning:

invalid escape sequence '\g'

<>:31: SyntaxWarning:

invalid escape sequence '\g'

<>:48: SyntaxWarning:

invalid escape sequence '\g'

C:\Users\yoe11\AppData\Local\Temp\ipykernel_18708\3299150170.py:31: SyntaxWarning:

invalid escape sequence '\g'

C:\Users\yoe11\AppData\Local\Temp\ipykernel_18708\3299150170.py:48: SyntaxWarning:

invalid escape sequence '\g'



## 2024

In [19]:
finance = df_financiamientos_2024_vallarta[['id','ingresos_rango']]
registros_0 = finance[finance['ingresos_rango'] == 0].shape[0]
print(registros_0)
df_2022 = finance.shape
print(df_2022)
#finance_2022_pachuca = finance_2022_pachuca[finance_2022_pachuca['ingresos_rango'] != 0]

24
(1180, 2)


In [20]:
# Convertir la columna 'ingresos_rango' a cadena de texto (str) 
#finance_2022['ingresos_rango'] = finance_2022['ingresos_rango'].astype(str)
finance.loc[:, 'ingresos_rango'] = finance['ingresos_rango'].astype(str)
# Reemplazar los valores de rango con el texto correspondiente utilizando replace
#finance_2022['ingresos_rango'] = finance_2022['ingresos_rango'].replace({
finance.loc[:, 'ingresos_rango'] = finance['ingresos_rango'].replace({
    
    '0': '0',
    '1': '≤ 2.60',
    '2': '2.61-4.00',
    '3': '4.01-6.00',
    '4': '6.01-9.00',
    '5': '9.01-12.00',
    '6': '> 12.00',
})
finance

C:\Users\yoe11\AppData\Local\Temp\ipykernel_18708\2444930754.py:3: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['1' '4' '6' ... '3' '4' '0']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.



,id,ingresos_rango
0,1,≤ 2.60
1,2,6.01-9.00
2,3,> 12.00
3,4,≤ 2.60
4,5,4.01-6.00
...,...,...
1175,1176,2.61-4.00
1176,1177,2.61-4.00
1177,1178,4.01-6.00
1178,1179,6.01-9.00


In [21]:
# Eliminar la fila donde rango_ingresos es '0'
finance = finance[finance['ingresos_rango'] != '0']

# Calcular el conteo de cada rango de ingresos
conteo_ingresos = finance['ingresos_rango'].value_counts().reset_index()
conteo_ingresos.columns = ['rango_ingresos', 'conteo']
# Calcular los porcentajes
total_registros = len(finance)
conteo_ingresos['porcentajes'] = round((conteo_ingresos['conteo'] / total_registros) * 100,2)
conteo_ingresos

,rango_ingresos,conteo,porcentajes
0,> 12.00,249,21.54
1,4.01-6.00,211,18.25
2,6.01-9.00,208,17.99
3,2.61-4.00,185,16.00
4,≤ 2.60,161,13.93
5,9.01-12.00,142,12.28


In [22]:
# Definir el diccionario de orden de rangos de ingresos
orden_rangos = {
    '≤ 2.60': 1,
    '2.61-4.00': 2,
    '4.01-6.00': 3,
    '6.01-9.00': 4,
    '9.01-12.00': 5,
    '> 12.00': 6
}

# Añadir una columna para el orden de los rangos
conteo_ingresos['orden'] = conteo_ingresos['rango_ingresos'].map(orden_rangos)
# Ordenar el DataFrame por el orden de los rangos
conteo_ingresos = conteo_ingresos.sort_values(by='orden')
# Eliminar la columna de orden
conteo_ingresos = conteo_ingresos.drop(columns=['orden'])
conteo_ingresos

,rango_ingresos,conteo,porcentajes
4,≤ 2.60,161,13.93
3,2.61-4.00,185,16.00
1,4.01-6.00,211,18.25
2,6.01-9.00,208,17.99
5,9.01-12.00,142,12.28
0,> 12.00,249,21.54


In [23]:
labels = conteo_ingresos['rango_ingresos']
values = conteo_ingresos['porcentajes']
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6']

fig = go.Figure()
fig.add_trace(go.Bar(
        x=labels, 
        y=values, 
        marker_color=colores,
        text=values,  # Texto que se mostrará en las barras (porcentaje)
        textposition='inside',  # Posición del texto (puede ser 'inside' o 'outside')
        texttemplate='%{text:.1f}%', 
    )
)    
fig.update_layout(
    #barmode='stack', 
    #title='Porcentaje de créditos en UMAS (2022)',
    #title_font_size = 22,
    yaxis=dict(
        #title='M', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y         
    ),  margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    legend=dict(
        yanchor="top",
        y=0.99,
    )
)

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets\graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
   # Configuración para desactivar herramientas no deseadas
    config = {
        'displaylogo': False,  # Ocultar el logo de Plotly
        'modeBarButtonsToRemove': [
            'toImage',       # Botón para guardar como imagen
            'select2d',      # Box select
            'lasso2d',       # Lasso select
            'resetScale2d',  # Reset Axes
        ]
    }
    # Exportar el gráfico con configuración
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html', config=config)

guardar_grafico_como_html(fig, 'g_bar_creditos_umas_2024', carpeta='assets\graficas')
fig.show()

<>:31: SyntaxWarning:

invalid escape sequence '\g'

<>:48: SyntaxWarning:

invalid escape sequence '\g'

<>:31: SyntaxWarning:

invalid escape sequence '\g'

<>:48: SyntaxWarning:

invalid escape sequence '\g'

C:\Users\yoe11\AppData\Local\Temp\ipykernel_18708\744023874.py:31: SyntaxWarning:

invalid escape sequence '\g'

C:\Users\yoe11\AppData\Local\Temp\ipykernel_18708\744023874.py:48: SyntaxWarning:

invalid escape sequence '\g'

